In [1]:
from ibpythonic import ibConnection, message
from ibapi.contract import Contract

from time import sleep, strftime
from datetime import datetime

import pandas as pd
import numpy as np

from tinydb import TinyDB, Query

from tqdm import tqdm_notebook as tqdm

In [2]:
def makeStkContract(contractTuple):
    newContract = Contract()
    newContract.symbol = contractTuple[0]
    newContract.secType = contractTuple[1]
    newContract.exchange = contractTuple[2]
    newContract.currency = contractTuple[3]
    return newContract

In [3]:
bar_data_list = []
busy = False
active_symbol = 0

In [4]:
def my_hist_data_handler(msg):
    global bar_data_list
    bar_data_list.append(msg.bar)

In [5]:
def my_hist_data_end_handler(msg):
    global bar_data_list
    global busy
    global active_symbol
        
    # Merge collected data into dataframe
    df = pd.DataFrame(columns=('date', 'open', 'high', 'low', 'close', 'volume'))
    for bar in bar_data_list:
        row = {'date': bar.date, 'open': bar.open, 'high': bar.high, 'low': bar.low, 'close': bar.close, 'volume': bar.volume}
        df = df.append(row, ignore_index=True)
    df.loc[:, 'date'] = pd.to_datetime(df.date, format='%Y%m%d') #'%Y%m%d  %H:%M:%S'
    df = df.set_index('date')

    # Write dataframe to csv file
    symbol_db = TinyDB('symbol_db.json')
    my_query = Query()
    result = symbol_db.search(my_query.Id == active_symbol)
    current_symbol = result[0]
    sy = current_symbol['Symbol']
    ex = current_symbol['Exchange']
    df.to_csv('data/' + sy + '_' + ex + '.csv')
    
    # write finished info to symbol database
    symbol_db.update({'Status': 'Finished'}, my_query.Id == active_symbol)
    symbol_db.close()
    
    bar_data_list = []
    active_symbol = []
    busy = False

In [6]:
def my_error_handler(msg):
    global bar_data_list
    global busy
    global active_symbol
    
    # write msg info to symbol database
    symbol_db = TinyDB('symbol_db.json')
    my_query = Query()
    symbol_db.update({'Status': msg.errorMsg}, my_query.Id == active_symbol)
    symbol_db.close()
    
    bar_data_list = []
    active_symbol = []
    busy = False

In [7]:
# file = open('symbollist.txt')
# symbols = file.read().splitlines()
# file.close()
# symbols = symbols[:3]

# symbols_df = pd.read_csv('symbol_database.csv', sep=';')
# symbols_df['Status'] = 'Untouched'
# symbols_df = symbols_df.loc[0:7, :]

# symbols = ['DAI', 'BMW', 'SAP', 'BAY']
# symbols = ['TSLA', 'ADBE', 'NVDA', 'AMD', 'INTL']

In [26]:
endtime = strftime('%Y%m%d %H:%M:%S')
start_id = 5000
end_id = 5100

In [27]:
con = ibConnection(port=7497)
con.register(my_hist_data_handler, message.historicalData)
con.register(my_hist_data_end_handler, message.historicalDataEnd)
con.register(my_error_handler, message.error)

True

In [29]:
con.connect()
    
for index in tqdm(range(start_id, end_id+1)):
    global busy
    global active_symbol
    
    # Get symbol data from symbol db
    symbol_db = TinyDB('symbol_db.json')
    my_query = Query()
    result = symbol_db.search(my_query.Id == index)
    symbol_db.close()
    current_symbol = result[0]
    
    # Skip symbol if data present
    if current_symbol['Status'] != 'untouched':
        continue
    
    # Create contract and request data
    contractTuple = (current_symbol['Symbol'], 'STK', current_symbol['Exchange'], current_symbol['Currency'])
    stkContract = makeStkContract(contractTuple)
    active_symbol = index
    busy = True
    con.reqHistoricalData(1,stkContract,endtime,"10 Y","1 day","MIDPOINT",1,1, keepUpToDate=False, chartOptions=[])
    
    # Wait for data download and storage
    while busy is True:
        sleep(0.2)
    sleep(0.2)

con.disconnect()

unhandled exception in EReader thread
Traceback (most recent call last):
  File "c:\users\marti\dev\_pyvenvs\latest36\lib\site-packages\ibapi\reader.py", line 34, in run
    data = self.conn.recvMsg()
  File "c:\users\marti\dev\_pyvenvs\latest36\lib\site-packages\ibapi\connection.py", line 99, in recvMsg
    buf = self._recvAllMsg()
  File "c:\users\marti\dev\_pyvenvs\latest36\lib\site-packages\ibapi\connection.py", line 119, in _recvAllMsg
    buf = self.socket.recv(4096)
OSError: [WinError 10038] An operation was attempted on something that is not a socket


True

In [25]:
symbol_db = TinyDB('symbol_db.json')
my_query = Query()
result = symbol_db.search(my_query.Status == 'Finished')
for item in result:
    print(item)
symbol_db.close()

{'Id': 1, 'Symbol': 'CNB', 'Name': 'Lyxor Euro Corporate Bond Ex Financials UCITS ETF', 'Currency': 'EUR', 'Exchange': 'FWB', 'Status': 'Finished'}
{'Id': 4, 'Symbol': 'X0BM', 'Name': 'Coba ETC -2 x Platinum Daily Short Index', 'Currency': 'EUR', 'Exchange': 'FWB', 'Status': 'Finished'}
{'Id': 6, 'Symbol': 'X0E2', 'Name': 'Coba ETC 2x Gasoil Daily Long Index', 'Currency': 'EUR', 'Exchange': 'FWB', 'Status': 'Finished'}
{'Id': 1003, 'Symbol': 'X04J', 'Name': 'Coba ETN 2x SPXF Daily Long', 'Currency': 'EUR', 'Exchange': 'FWB', 'Status': 'Finished'}
{'Id': 1004, 'Symbol': '0LJH', 'Name': 'Boost Short USD Long EUR 5x Daily ETP', 'Currency': 'EUR', 'Exchange': 'FWB', 'Status': 'Finished'}
{'Id': 1010, 'Symbol': '4RUA', 'Name': 'ETFS 2X Daily Long Lean Hogs', 'Currency': 'EUR', 'Exchange': 'FWB', 'Status': 'Finished'}
{'Id': 1012, 'Symbol': 'X0CK', 'Name': 'Coba ETC -3x Gold Daily Short', 'Currency': 'EUR', 'Exchange': 'FWB', 'Status': 'Finished'}
{'Id': 1014, 'Symbol': 'OD7H', 'Name': 'ETFS

In [57]:
 con.disconnect()

False

# Enhancements

### Speedup
- dict of symbols with ascending numbers
- download data one after another, symbol-number is id for downloading
- write received data to file as fast as possible, filename is symbol-number
- download next symbol as soon as data is in file
- in parallel load finished data from files, convert to dataframes and save as csv

In [ ]:
with open('your_file.txt', 'w') as f:
    for item in my_list:
        f.write("%s\n" % item)

# or

outfile.write("\n".join(itemlist))

# or

import pickle

with open('outfile', 'wb') as fp:
    pickle.dump(itemlist, fp)
    
with open ('outfile', 'rb') as fp:
    itemlist = pickle.load(fp)
    
# or

open("bla.txt", "wb").write(''.join(random.choice(string.ascii_lowercase) for i in xrange(10**7)))

### Extra features for downloading
- check for existing data
- start time for downloading is end time of data. last candle of data is considered invalid
- check for splits/mergers

### Symbol in multiple exchanges
- if symbol is multiple times in list
    - load data from primary exchange if ppossible
    - if primary is not available, load from exchange with highest average (price-)volume


### Symbol database
- pull symbols from ib webpage
- create symbol database
- pull symbol data from ib
- abillity to create own univestes (=symbol lists) based on symbol data

### Storage of price data
- store prica data in CSV per symbol per timframe
- store price data in database? sqlite, as handeled in one file?
- cloud storage of data

### Misc
- 